# Проект: Проверка гипотезы о различии времени активности пользователей из Москвы и Санкт-Петербурга

- Автор: Елагина Ксения

## Введение

### Цели и задачи проекта

**Цель проекта:**
Проверить гипотезу о том, что пользователи из Санкт-Петербурга проводят больше времени за чтением и прослушиванием книг в приложении Яндекс Книги, чем пользователи из Москвы.

**Задачи проекта:**

- Загрузить и предобработать данные.
- Проверить наличие дубликатов.
- Рассчитать основные статистические показатели для каждой группы пользователей.
- Провести проверку гипотезы.
- Сделать выводы.

## Загрузка данных и знакомство с ними


In [1]:
# Импортируем библиотеки
import pandas as pd
from scipy import stats as st
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize

In [2]:
# Выгружаем данные
df = pd.read_csv('/datasets/yandex_knigi_data.csv')

In [3]:
# Выведем информацию о датафрейме:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  8784 non-null   int64  
 1   city        8784 non-null   object 
 2   puid        8784 non-null   int64  
 3   hours       8784 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 274.6+ KB


Проверим наличие дубликатов в идентификаторах пользователей. Сравним размеры групп, их статистики и распределение.


In [4]:
# Найдем количество дубликатов в идентификаторах пользователей
df['puid'].duplicated().sum()

244

In [5]:
# Удалим дубликаты
df_unique = df.drop_duplicates(subset='puid', keep=False)

# Проверим удаление
df_unique['puid'].duplicated().sum()

0

Датасет df содержит 8784 столбца и 4 строки, в которых представлена информация о пользователях приложения Яндекс Книги из Москвы и Санкт-Петербурга.

После первичного анализа данных можно сделать следующие выводы:

- Столбец `Unnamed: 0` содержит в себе индексы, которые и так есть в таблице, поэтому этот столбец можно удалить. Тем более название столбца непонятно и неинформативно.
- Данные полные, пропусков нет.
- Типы данных соответствуют.
- В столбце с идентификаторами пользователей было найдено 244 дубликата. Возможно, они возникли из-за того, что юзеры пользовались сервисом в разных городах. Эти дубликаты были удалены.

In [6]:
# Создадим 2 таблицы с данными о Москве и СПБ, чтобы их сравнить
df_msc = df_unique[df_unique['city']=='Москва']
df_msc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5990 entries, 0 to 6233
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  5990 non-null   int64  
 1   city        5990 non-null   object 
 2   puid        5990 non-null   int64  
 3   hours       5990 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 234.0+ KB


In [7]:
df_spb = df_unique[df_unique['city']=='Санкт-Петербург']
df_spb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2306 entries, 6234 to 8783
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  2306 non-null   int64  
 1   city        2306 non-null   object 
 2   puid        2306 non-null   int64  
 3   hours       2306 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 90.1+ KB


Пользователей из Москвы намного больше, чем из Санкт-Петербурга (5990 из Москвы и 2306 из СПб).

## Проверка гипотезы в Python

Попробуем статистически доказать гипотезу.

- **Нулевая гипотеза H₀**: средняя активность пользователей из Москвы и Санкт-Петербурга не различается.

- **Альтернативная гипотеза H₁**: пользователи из Санкт-Петербурга проводят в среднем больше времени за чтением и прослушиванием книг в приложении, чем пользователи из Москвы, и это различие статистически значимо.

Расчитаем необходимый размер выборки с помощью калькулятора. Возьмем за базовый уровень - 10%, а mde - 3%.

https://www.evanmiller.org/ab-testing/sample-size.html#!10;80;5;3;0

Соответственно, необходимый размер групп равен 1629 человек. В обеих группах взято большее количество пользователей, поэтому можно проводить статистический тест.

Рассмотрим статистику рассматриваемой метрики для каждой группы:

In [8]:
df_msc['hours'].describe(percentiles=[0.25, 0.75, 0.99])

count    5990.000000
mean       10.848192
std        36.925622
min         0.000022
25%         0.057042
50%         0.888232
75%         5.933439
99%       154.688119
max       857.209373
Name: hours, dtype: float64

In [9]:
df_spb['hours'].describe(percentiles=[0.25, 0.75, 0.99])

count    2306.000000
mean       11.264433
std        39.831755
min         0.000025
25%         0.060173
50%         0.875355
75%         6.138424
99%       146.847159
max       978.764775
Name: hours, dtype: float64

Описание статистики показывает, что у обеих групп ненормальное распределение значений: среднее сильно отличается от медианы (у Москвы медиана равняется 10.8 часов, среднее - 0.9 ч, а у СПб 11.3ч и 0.9ч соответственно). Также можно увидеть большое количество выбросов у данных: 99 перцентили имеют невысокие значения по сравнению с максимальным (у Москвы 99% равен 154.7ч, макс - 857.2ч, у СПб - 146.8ч и 978.8ч соответственно).

Для того, чтобы решить, какой применить т-тест, проверим распределение данных у обеих групп с помощью теста Шапиро–Уилка.

**Нулевая гипотеза** - данные распределены нормально

**Альтернативная гипотеза** - данные не нормальны

In [10]:
from scipy.stats import shapiro

# уровень значимости
alpha = 0.05

# проведем тест на данных Москвы
stat, p_value = shapiro(df_msc['hours'])

if p_value>alpha:
    print(f'p-value статистики Шапиро-Уилка = {p_value}')
    print('Распределение группы Москвы можно считать нормальным.')
else:
    print(f'p-value статистики Шапиро-Уилка ={p_value}')
    print('Данные группы Москвы распределены ненормально.')

p-value статистики Шапиро-Уилка =0.0
Данные группы Москвы распределены ненормально.


D:\dd\Python\Python310\lib\site-packages\scipy\stats\_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [11]:
# проведем тест на данных СПб
stat, p_value = shapiro(df_spb['hours'])

if p_value>alpha:
    print(f'p-value статистики Шапиро-Уилка = {p_value}')
    print('Распределение группы СПБ можно считать нормальным.')
else:
    print(f'p-value статистики Шапиро-Уилка ={p_value}')
    print('Данные группы Спб распределены ненормально.')

p-value статистики Шапиро-Уилка =0.0
Данные группы Спб распределены ненормально.


У обеих групп распределение ненормальное, есть выбросы, поэтому необходимо провести статистический тест Уэлча.

In [12]:
# выберем целевые метрики
metric_a = df_msc['hours']
metric_b = df_spb['hours']

In [13]:
from scipy.stats import ttest_ind

# уровень значимости
alpha = 0.05

# проведем тест Уэлча
stat_welch_ttest, p_value_welch_ttest = ttest_ind(metric_a,
                       metric_b,
                       alternative = 'less',
                       equal_var=False
                      )
if p_value_welch_ttest>alpha:
    print(f'p-value теста Уэлча = {round(p_value_welch_ttest, 2)}')
    print('Нулевая гипотеза находит подтверждение! Выборочные средние в группах A и B равны')
    print('Средняя активность пользователей в обеих выборках не различается')
else:
    print(f'p-value теста Уэлча = {round(p_value_welch_ttest, 2)}')
    print('Нулевая гипотеза не находит подтверждения! Выборочные средние в группах A и B не равны, и в группе B больше')
    print('Средняя активность пользователей в обеих выборках различается')

p-value теста Уэлча = 0.33
Нулевая гипотеза находит подтверждение! Выборочные средние в группах A и B равны
Средняя активность пользователей в обеих выборках не различается


## Аналитическая записка

**Выбранный тип теста и уровень статистической значимости:**

Для проверки гипотезы был выбран односторонний t-тест Уэлча для независимых выборок с параметром alternative = 'less'. Это соответствует проверке гипотезы:

**Нулевая гипотеза (H₀)**: среднее время активности пользователей из Москвы больше или равно среднему времени активности пользователей из Санкт-Петербурга.

**Альтернативная гипотеза (H₁)**: среднее время активности пользователей из Москвы меньше, чем у пользователей из Санкт-Петербурга.

Уровень статистической значимости был выбран стандартный — 0.05.

**Результаты теста:**

p-value теста Уэлча = 0.33

**Вывод и интерпретация результатов:**

Полученное значение p-value значительно превышает выбранный уровень значимости (0.33 > 0.05), поэтому нет оснований отклонить нулевую гипотезу. Это значит, что в выборках пользователей из Москвы и Санкт-Петербурга нет статистически значимых различий в среднем времени активности в приложении.

**Возможные объяснения полученных результатов:**

Поведение пользователей в обеих городах может быть схожим: аудитория Яндекс Книг в крупных городах показывает одинаковую вовлечённость независимо от региона.

Возможно, разница между группами существует, но она слишком мала, чтобы быть обнаруженной на имеющемся объёме данных или при текущем уровне разброса значений.

**Рекомендации:**
- Считать пользовательскую активность в Москве и Санкт-Петербурге статистически одинаковой при дальнейшем анализе и сегментации.  
- Для выявления более тонких различий рассмотреть расширение выборки и дополнительное исследование факторов, влияющих на время активности (например, возраст, устройство, жанры контента).  